In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from config.config import BASE_URL, OUTPUT_FILE
from logger.logger import logger

headers = {'User-Agent': 'Mozilla/5.0'}

def get_unique_item_ids(max_pages=200):
    """Fetch unique item IDs from paginated pages with proper logging."""
    item_ids = set()
    page = 1
    has_more_pages = True
    
    logger.info(f"Starting to scrape item IDs (max pages: {max_pages})")
    
    while has_more_pages:
        if max_pages and page > max_pages:
            logger.info(f"Reached max pages limit ({max_pages})")
            break
            
        url = f"{BASE_URL}?page={page}"
        logger.debug(f"Fetching page {page}: {url}")
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                logger.error(f"Failed to fetch page {page}. Status code: {response.status_code}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            
            # Find all item links and extract IDs from hrefs
            current_page_ids = set()
            for a in soup.select("a.products-i__link"):
                href = a.get("href")
                if href:
                    match = re.search(r'/autos/(\d+)', href)
                    if match:
                        item_id = match.group(1)
                        current_page_ids.add(item_id)
            
            # Check if we found any new items on this page
            if not current_page_ids:
                logger.info(f"No items found on page {page}, stopping pagination")
                has_more_pages = False
            else:
                new_items = current_page_ids - item_ids
                if not new_items:
                    logger.info(f"No new items found on page {page}, stopping pagination")
                    has_more_pages = False
                else:
                    logger.info(f"Found {len(new_items)} new items on page {page}")
                    item_ids.update(current_page_ids)
                    page += 1
                    
        except requests.RequestException as e:
            logger.error(f"Network error fetching page {page}: {str(e)}", exc_info=True)
            break
        except Exception as e:
            logger.error(f"Unexpected error processing page {page}: {str(e)}", exc_info=True)
            break
    logger.info(f"Finished scraping. Found {len(item_ids)} unique items")
    return item_ids    

# Run the function and print results
item_ids = get_unique_item_ids()
links = list(item_ids)
print(links)
print(len(links))

2025-06-22 17:27:56 - scraper - INFO - Starting to scrape item IDs (max pages: 200)
2025-06-22 17:27:57 - scraper - INFO - Found 36 new items on page 1
2025-06-22 17:28:00 - scraper - INFO - Found 36 new items on page 2
2025-06-22 17:28:03 - scraper - INFO - Found 35 new items on page 3
2025-06-22 17:28:04 - scraper - INFO - Found 36 new items on page 4
2025-06-22 17:28:07 - scraper - INFO - Found 35 new items on page 5
2025-06-22 17:28:09 - scraper - INFO - Found 36 new items on page 6
2025-06-22 17:28:12 - scraper - INFO - Found 34 new items on page 7
2025-06-22 17:28:13 - scraper - INFO - Found 34 new items on page 8
2025-06-22 17:28:15 - scraper - INFO - Found 36 new items on page 9
2025-06-22 17:28:16 - scraper - INFO - Found 35 new items on page 10
2025-06-22 17:28:18 - scraper - INFO - Found 35 new items on page 11
2025-06-22 17:28:21 - scraper - INFO - Found 36 new items on page 12
2025-06-22 17:28:25 - scraper - INFO - Found 36 new items on page 13
2025-06-22 17:28:26 - scrape

['9546767', '9470036', '9543639', '9537824', '9552866', '8689677', '9498872', '9396588', '9503176', '7064479', '9555484', '9490810', '9549779', '9556200', '9459731', '9438951', '8239794', '9113091', '9556173', '9555887', '9520880', '9526938', '9516944', '9556311', '9024938', '9552226', '9363989', '9518266', '9131126', '9486745', '9540389', '9549320', '9475951', '9487082', '8758227', '9463022', '9555743', '9553751', '9554221', '9551497', '9507894', '9515924', '9555893', '9475208', '9556586', '9536832', '9525840', '9523142', '9541190', '9473165', '9541048', '9318187', '9326789', '9556261', '9504674', '9417013', '9554004', '9502785', '9511741', '9282359', '9384400', '9451287', '9532843', '9539461', '7467609', '9376425', '9170856', '9488758', '9305753', '6757698', '9452402', '9223310', '9448309', '9554737', '9518472', '9556252', '9458750', '9432978', '9549044', '9071534', '9238904', '9465481', '9556542', '8858060', '9522821', '9346021', '9555886', '9556151', '9382567', '9556558', '9543113'

In [ ]:
def extract_item_data(link):
    """Extract detailed item data from individual product page with comprehensive logging"""
    url = "https://turbo.az/autos/" + link
    logger.debug(f"Starting to extract data from: {url}")
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            logger.error(f"Failed to fetch {url} - Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract item ID
        item_id = link.split("/")[-1]
        logger.debug(f"Processing item ID: {item_id}")

        # Extract amount and currency using meta tags
        meta_tag = soup.find('title')

        # Extract content attribute
        content = meta_tag.get('content')

        # Use regex to find price followed by either USD or AZN (case-insensitive)
        match = re.search(r'(\d[\d\s]*\d)\s+(USD|AZN)', content, re.IGNORECASE) 

        if match:
            amount = match.group(1).replace(' ', '')  # Remove space
            currency = match.group(2).upper()  # Get the currency type

        logger.debug(f"Extracted price: {amount} {currency}")

        # Extract property fields dynamically
        properties = {}
        prop_div = soup.find("div", class_="product-properties__column")
        if prop_div:
            for prop in prop_div.find_all("div", class_="product-properties__i"):
                try:
                    name = prop.find("label", class_="product-properties__i-name").get_text(strip=True)
                    value = prop.find("span", class_="product-properties__i-value").get_text(strip=True)
                    properties[name] = value
                    logger.debug(f"Found property: {name} = {value}")
                except Exception as e:
                    logger.warning(f"Failed to extract property: {str(e)}", exc_info=True)

        # Construct result dictionary
        item_data = {
            'item_id': item_id,
            'amount': amount,
            'currency': currency,
        }

        # Add all found properties dynamically
        item_data.update(properties)
        
        logger.info(f"Successfully processed item: {item_id}")
        return item_data

    except requests.RequestException as e:
        logger.error(f"Network error while processing {url}: {str(e)}", exc_info=True)
        return None
    except Exception as e:
        logger.error(f"Unexpected error processing {url}: {str(e)}", exc_info=True)
        return None


def scrape_all_items():
    """Main scraping function that processes all items with progress tracking"""
    if not links:
        logger.warning("No links to process - empty list provided")
        return

    results = []
    logger.info(f"Starting to process {len(links)} items")

    for idx, link in enumerate(links, 1):
        logger.info(f"[{idx}/{len(links)}] Processing: {link}")
        data = extract_item_data(link)
        if data:
            results.append(data)
            logger.debug(f"Added item {data.get('item_id')} to results")
        else:
            logger.warning(f"Failed to process item: {link}")

    # Save to JSON
    try:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        logger.info(f"Successfully saved {len(results)} items to {OUTPUT_FILE}")
        if len(results) != len(links):
            logger.warning(f"Processed {len(results)} out of {len(links)} items ({(len(results)/len(links))*100:.1f}% success rate)")
    except Exception as e:
        logger.error(f"Failed to save data to {OUTPUT_FILE}: {str(e)}", exc_info=True)


# Run the scraper
scrape_all_items()

2025-06-18 17:09:46 - scraper - INFO - Starting to process 5584 items
2025-06-18 17:09:46 - scraper - INFO - [1/5584] Processing: 9319487
2025-06-18 17:09:47 - scraper - INFO - Successfully processed item: 9319487
2025-06-18 17:09:47 - scraper - INFO - [2/5584] Processing: 9543155
2025-06-18 17:09:48 - scraper - INFO - Successfully processed item: 9543155
2025-06-18 17:09:48 - scraper - INFO - [3/5584] Processing: 8509118
2025-06-18 17:09:48 - scraper - INFO - Successfully processed item: 8509118
2025-06-18 17:09:48 - scraper - INFO - [4/5584] Processing: 9424422
2025-06-18 17:09:49 - scraper - INFO - Successfully processed item: 9424422
2025-06-18 17:09:49 - scraper - INFO - [5/5584] Processing: 9545795
2025-06-18 17:09:49 - scraper - INFO - Successfully processed item: 9545795
2025-06-18 17:09:49 - scraper - INFO - [6/5584] Processing: 9437382
2025-06-18 17:09:50 - scraper - INFO - Successfully processed item: 9437382
2025-06-18 17:09:50 - scraper - INFO - [7/5584] Processing: 913213